In [13]:
!pip3 install --upgrade ultralytics

561.61s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [10]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.193 🚀 Python-3.11.1 torch-2.0.0 CPU (Apple M1)
Setup complete ✅ (8 CPUs, 16.0 GB RAM, 223.1/926.4 GB disk)


# Split images to 80:20

In [11]:
import os
import shutil
import random

# !pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
        if filex == 'classes':
            continue
        shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
        shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
        if filex == 'classes':
            continue
        # print("running")
        shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
        shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag
train_test_split('/content/drive/MyDrive/yolov8/data/') ### without negative images
# train_test_split('./data/','./negative_images/') ### negative images

# Train model

In [ ]:
# from ultralytics import YOLO

# # Load a model
# model = YOLO("yolov8l.pt")

# # Train the model with 2 GPUs
# results = model.train(data='data.yaml', epochs=400, batch=16, imgsz=640, device='mps', project="training_results", name="pharmacy100b16" )


In [ ]:
def train_yolo():
  !yolo task=detect mode=train model=yolov8l.pt data=data.yaml epochs=400 imgsz=640 batch=16 project=training_results name=pharmacy400b16 patience=0 device=mps

## recommend to train in colab, is better
# train_yolo()

In [ ]:
# from ultralytics import YOLO

# # Load a pretrained YOLOv8n model
# model = YOLO('best.pt')

# # Run inference on 'bus.jpg' with arguments
# result = model.predict('test_images/IMG_1607.JPG', save=True, conf=0.6, line_width=2, save_txt=True)


# Predict images

In [28]:
def detect_img():
  !yolo task=detect mode=predict model=best.pt conf=0.6 source=test_images/IMG_1607.JPG save=True save_txt=True line_width=2

# detect_img()

In [29]:
def read_file(counter):
  results = []
  ans = []
  names = ['Allergis', 'Blackmores', 'Grafadex', 'Heamovit', 'NatC', 'Rhinalax']

  with open(f'runs/detect/predict{counter}/labels/IMG_1607.txt') as f:
      lines = f.readlines()
      for i in lines:
        results.append(i[0])
  # print(results)

  for i in range(6):
    count = results.count(str(i))
    if count <= 1:
      ans.append('You have '+ names[i] + ' remain(s) ' + str(count) + '\n')

  message = 'Hello Boom' + '\n' + ''.join(ans) #Set your message here!

  # print(ans)
  return message

In [30]:
import requests

def linenotify(message):
  url = 'https://notify-api.line.me/api/notify'
  token = 'Y3Vtrey3Swmmuu8Ii8DkUCRPiGreLrOODDDzyVZlnWN' # Line Notify Token
  # img = {'imageFile': open('/content/drive/MyDrive/EBP_9.21.jpg','rb')} #Local picture File
  data = {'message': message}
  headers = {'Authorization':'Bearer ' + token}
  session = requests.Session()
  session_post = session.post(url, headers=headers, data =data) #files=img
  print(session_post.text)


In [31]:
from datetime import datetime
import pytz

counter = ''

tz_BK = pytz.timezone('Asia/Bangkok')
while True:
  datetime_BK = datetime.now(tz_BK)
  if datetime_BK.strftime("%T.%f")[:-3] == "13:47:00.000":
    detect_img()
    message = datetime_BK.strftime("%F %T.%f")[:-3] + "\n"
    message = message + read_file(counter)
    linenotify(message)
    if counter == '':
      counter = 2
    else:
      counter += 1
    print(counter)



1264.18s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
Ultralytics YOLOv8.0.193 🚀 Python-3.11.1 torch-2.0.0 CPU (Apple M1)
Model summary (fused): 268 layers, 43611234 parameters, 0 gradients, 164.8 GFLOPs

image 1/1 /Users/boomctr/Documents/Visual code/Senier Project/test_images/IMG_1607.JPG: 480x640 6 Heamovits, 2 NatCs, 6 Rhinalaxs, 684.3ms
Speed: 5.4ms preprocess, 684.3ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict2
1 label saved to runs/detect/predict2/labels
💡 Learn more at https://docs.ultralytics.com/modes/predict
{"status":200,"message":"ok"}
2


KeyboardInterrupt: 